# Introdução

Esse notebook traz as analises pedidas na disciplina Biologia Evolutiva - Bio507

Neste tutorial vamos utilizar a a linguagem Python 2.7 e os pacotes numpy, pandas, dendropy e matplotlib

## Leitura de dados

In [3]:
import numpy as np
import pandas as pd
import dendropy  as dp
import matplotlib as mpl

Primeiro vamos ler os dados usando o pandas.
A função read_csv cria um data.frame com os dados.

In [4]:
import numpy as np
import pandas as pd
import dendropy  as dp
import matplotlib as mpl

dados_brutos = pd.read_csv("./dados.csv")
num_traits = 4
traits = dados_brutos.columns[:num_traits]
dados_brutos.head(10)

,UMERO,ULNA,FEMUR,TIBIA,ESPECIE
0,3.214619,8.036072,13.517787,19.011308,A
1,5.222875,10.838375,15.547144,21.017178,A
2,5.193021,11.911770,17.247190,22.431903,A
3,6.547400,11.293085,14.890399,20.177057,A
4,4.724383,9.897135,14.811035,19.597533,A
5,4.630147,9.441174,15.247539,19.259374,A
6,6.771560,11.905400,15.571640,22.067775,A
7,5.229635,9.881575,13.987716,19.058813,A
8,4.786389,10.020607,15.676585,19.628285,A
9,4.171654,8.998204,15.523059,20.066215,A


Podemos usar a função describe para ter um resumo global, olhando para médias, desvio padrão e quartis.

In [5]:
dados_brutos.describe()

,UMERO,ULNA,FEMUR,TIBIA
count,300.000000,300.000000,300.000000,300.000000
mean,8.329141,16.710483,22.099096,29.439649
std,2.290569,4.370596,4.484063,5.912966
min,2.720256,7.145574,13.307339,18.199906
25%,6.752338,14.004285,19.904228,26.598481
50%,8.577857,16.742268,21.910598,28.775488
75%,10.197404,20.604040,26.021909,34.724381
max,13.067744,24.320296,30.659812,39.132379


Podemos também usar a função groupby para aplicar funções em sub conjuntos dos dados, agrupando pela coluna ESPECIE

In [6]:
dados_brutos.groupby('ESPECIE').describe()

FEMUR      TIBIA       ULNA      UMERO
ESPECIE                                                  
A       count  60.000000  60.000000  60.000000  60.000000
        mean   15.111193  20.179938   9.949203   4.948718
        std     1.001978   1.033350   1.042259   0.925434
        min    13.307339  18.199906   7.145574   2.720256
        25%    14.418396  19.531194   9.439593   4.460753
        50%    15.047091  20.018262   9.961670   4.951055
        75%    15.583092  20.841174  10.626844   5.450730
        max    17.963502  22.431903  11.990388   6.771560
B       count  60.000000  60.000000  60.000000  60.000000
        mean   25.583995  34.039911  17.043157   8.449116
        std     0.959801   1.026290   0.976311   1.043582
        min    23.771423  31.814285  15.259521   5.956538
        25%    24.942347  33.298108  16.489761   7.780216
        50%    25.467266  34.001643  16.742268   8.407886
        75%    26.215958  34.762250  17.629024   9.257750
        max    27.800655  36.145970  19.498368  10.353900
C       count  60.000000  60.000000  60.000000  60.000000
        mean   21.545636  28.411278  20.234410  10.086848
        std     1.093285   0.954835   0.967816   0.992821
        min    19.100210  26.204498  18.536947   7.915180
        25%    20.828731  27.711861  19.513071   9.368775
        50%    21.689081  28.312005  20.082332  10.029947
        75%    22.222772  29.228187  20.898986  10.751811
        max    24.249630  30.351911  22.437891  12.299911
D       count  60.000000  60.000000  60.000000  60.000000
        mean   27.701677  37.039024  21.917249  10.764505
        std     1.092377   0.954970   0.999072   0.997346
        min    25.050549  34.571030  19.323098   8.294274
        25%    27.085591  36.360565  21.359714  10.082454
        50%    27.638454  37.123435  21.845116  10.749358
        75%    28.133082  37.622279  22.445897  11.335582
        max    30.659812  39.132379  24.320296  13.067744
E       count  60.000000  60.000000  60.000000  60.000000
        mean   20.552979  27.528095  14.408397   7.396516
        std     0.954628   1.002057   0.800716   0.984802
        min    17.708506  25.826498  12.339996   5.426811
        25%    20.119694  26.641851  14.004285   6.797075
        50%    20.474181  27.393652  14.384812   7.340141
        75%    21.161520  28.284489  14.957284   7.970026
        max    22.462786  29.440323  16.104371  10.177678

Vamos calcular as médias, desvios padrão e coeficiantes de variação por caractere por espécie.

In [7]:
medias = dados_brutos.groupby('ESPECIE').mean()
medias

,UMERO,ULNA,FEMUR,TIBIA
ESPECIE,,,,
A,4.948718,9.949203,15.111193,20.179938
B,8.449116,17.043157,25.583995,34.039911
C,10.086848,20.234410,21.545636,28.411278
D,10.764505,21.917249,27.701677,37.039024
E,7.396516,14.408397,20.552979,27.528095


In [8]:
std = dados_brutos.groupby('ESPECIE').std()
std

,UMERO,ULNA,FEMUR,TIBIA
ESPECIE,,,,
A,0.925434,1.042259,1.001978,1.033350
B,1.043582,0.976311,0.959801,1.026290
C,0.992821,0.967816,1.093285,0.954835
D,0.997346,0.999072,1.092377,0.954970
E,0.984802,0.800716,0.954628,1.002057


In [9]:
cv = std/medias
cv

,UMERO,ULNA,FEMUR,TIBIA
ESPECIE,,,,
A,0.187005,0.104758,0.066307,0.051207
B,0.123514,0.057285,0.037516,0.030150
C,0.098427,0.047830,0.050743,0.033608
D,0.092651,0.045584,0.039434,0.025783
E,0.133144,0.055573,0.046447,0.036401


# Visualizando os dados

Vamos usar o pacote matplotlib para fazer gráficos diagnósticos dos dados.

In [10]:
# histogramas

In [11]:
# scatter plots

In [12]:
# boxplots

# Calculando matrizes de covariância

Vamos calcular as matrizes de covariância e correlação por espécie, além das médias, e agrupá-las num dicionário.

In [13]:
cov_matrices = dados_brutos.groupby('ESPECIE').apply(lambda x: x.cov())
cor_matrices = dados_brutos.groupby('ESPECIE').apply(lambda x: x.corr())
especies_labels = list(pd.unique(dados_brutos['ESPECIE']))

In [14]:
cov_matrices

UMERO      ULNA     FEMUR     TIBIA
ESPECIE                                              
A       UMERO  0.856428  0.775719  0.216278  0.275482
        ULNA   0.775719  1.086305  0.317295  0.357705
        FEMUR  0.216278  0.317295  1.003959  0.796279
        TIBIA  0.275482  0.357705  0.796279  1.067811
B       UMERO  1.089062  0.441458  0.173216  0.197140
        ULNA   0.441458  0.953183  0.037394  0.137664
        FEMUR  0.173216  0.037394  0.921217  0.679280
        TIBIA  0.197140  0.137664  0.679280  1.053271
C       UMERO  0.985693  0.757909  0.016274  0.328934
        ULNA   0.757909  0.936668  0.085370  0.336382
        FEMUR  0.016274  0.085370  1.195272  0.173524
        TIBIA  0.328934  0.336382  0.173524  0.911711
D       UMERO  0.994699  0.776167  0.562616  0.558410
        ULNA   0.776167  0.998145  0.790311  0.636777
        FEMUR  0.562616  0.790311  1.193287  0.605724
        TIBIA  0.558410  0.636777  0.605724  0.911968
E       UMERO  0.969835 -0.096421  0.020960  0.360699
        ULNA  -0.096421  0.641145  0.010365  0.127960
        FEMUR  0.020960  0.010365  0.911314  0.248547
        TIBIA  0.360699  0.127960  0.248547  1.004119

In [15]:
cor_matrices

UMERO      ULNA     FEMUR     TIBIA
ESPECIE                                              
A       UMERO  1.000000  0.804235  0.233244  0.288072
        ULNA   0.804235  1.000000  0.303829  0.332125
        FEMUR  0.233244  0.303829  1.000000  0.769060
        TIBIA  0.288072  0.332125  0.769060  1.000000
B       UMERO  1.000000  0.433286  0.172934  0.184068
        ULNA   0.433286  1.000000  0.039905  0.137392
        FEMUR  0.172934  0.039905  1.000000  0.689601
        TIBIA  0.184068  0.137392  0.689601  1.000000
C       UMERO  1.000000  0.788776  0.014993  0.346984
        ULNA   0.788776  1.000000  0.080683  0.364008
        FEMUR  0.014993  0.080683  1.000000  0.166225
        TIBIA  0.346984  0.364008  0.166225  1.000000
D       UMERO  1.000000  0.778955  0.516409  0.586296
        ULNA   0.778955  1.000000  0.724150  0.667422
        FEMUR  0.516409  0.724150  1.000000  0.580647
        TIBIA  0.586296  0.667422  0.580647  1.000000
E       UMERO  1.000000 -0.122277  0.022295  0.365514
        ULNA  -0.122277  1.000000  0.013559  0.159479
        FEMUR  0.022295  0.013559  1.000000  0.259826
        TIBIA  0.365514  0.159479  0.259826  1.000000

Podemos também acessar as matrizes pelo nome da espécie:

In [14]:
cor_matrices.T['C']

,UMERO,ULNA,FEMUR,TIBIA
UMERO,1.000000,0.788776,0.014993,0.346984
ULNA,0.788776,1.000000,0.080683,0.364008
FEMUR,0.014993,0.080683,1.000000,0.166225
TIBIA,0.346984,0.364008,0.166225,1.000000


Ou fazer um heatplot de todas elas

In [16]:
#heat plots

# Analise de variância

Calculando as matrizes dentro de grupos, entre grupos e total

In [16]:
# Matrizes dentro e entre grupos

Loading required package: plyr


# Comparando as matrizes

In [17]:
# RandomSkewers

# Filogenia

Vamos incluir uma filogenia e calcular estados ancestrais

In [20]:
dp.Tree.get_from_string("4(E, 3(2(C, B),1(A,D)))", "newick")
#tree.print_plot(display_width = 50, show_internal_node_labels = True, leaf_spacing_factor = 4)

NewickReaderMalformedStatementError: Error parsing data source on line 1 at column 2: Malformed tree statement

Esse código usa as matrizes e médias calculadas anteriormente, junto com os tamanhos amostrais, para calcular valores ponderados para todos os nós da filogenia. 

A conta realizada para médias e matrizes é uma simples média ponderada.

In [23]:
get_node_name = lambda n: str(n.label or n.taxon or None)
nodes = [get_node_name(n) for n in tree.nodes()]

node_matrices = {}
node_sample_size = {}
for sp in especies_labels:
    new_matrix = np.array(cov_matrices.T[sp])
    node_matrices[sp] = new_matrix
    node_sample_size[sp] = dados_brutos[dados_brutos['ESPECIE'] == sp].shape[0]

# Tirando quem nao esta na filogenia e trocando os keys

node_means = {}

for sp in especies_labels:
    if tree.find_node_with_taxon_label(sp):
        new_key = get_node_name(tree.find_node_with_taxon_label(sp))
        node_means[new_key] = medias.T[sp]
        node_sample_size[new_key] = node_sample_size.pop(sp)
        node_matrices[new_key] = node_matrices.pop(sp)
    else:
        node_matrices.pop(sp)
        node_sample_size.pop(sp)

# Função que recebe uma lista de filhos e calcula a matriz, média e tamanho amostral pro ancestral

def ancestral_mean(child_labels):
    new_matrix = np.zeros((num_traits, num_traits))
    sample = 0
    new_mean = np.zeros(num_traits)
    for child in child_labels:
        node = get_node_name(child)
        new_matrix = new_matrix +\
            node_sample_size[node] * node_matrices[node]
        sample = sample + node_sample_size[node]
        new_mean = new_mean + node_sample_size[node] * node_means[node]
    new_matrix = new_matrix/sample
    new_mean = new_mean/sample
    return new_matrix, sample, new_mean

# Calculando as matrizes e tamanhos amostrais para todos os nós

for n in tree.postorder_node_iter():
    if get_node_name(n) not in node_matrices:
        node_matrices[get_node_name(n)], node_sample_size[get_node_name(n)], node_means[get_node_name(n)] = ancestral_mean(n.child_nodes())

Isso resulta num dicionário, chamado node_matrices, com todas as matrizes para todos os nós.

In [24]:
node_matrices

{'1': array([[ 0.92556338,  0.77594269,  0.38944731,  0.4169458 ],
       [ 0.77594269,  1.04222473,  0.55380272,  0.49724078],
       [ 0.38944731,  0.55380272,  1.0986233 ,  0.70100138],
       [ 0.4169458 ,  0.49724078,  0.70100138,  0.98988962]]),
 '2': array([[ 1.03737758,  0.59968352,  0.09474498,  0.26303718],
       [ 0.59968352,  0.94492564,  0.06138205,  0.23702306],
       [ 0.09474498,  0.06138205,  1.05824473,  0.42640187],
       [ 0.26303718,  0.23702306,  0.42640187,  0.98249091]]),
 '3': array([[ 0.98147048,  0.68781311,  0.24209615,  0.33999149],
       [ 0.68781311,  0.99357518,  0.30759238,  0.36713192],
       [ 0.24209615,  0.30759238,  1.07843402,  0.56370163],
       [ 0.33999149,  0.36713192,  0.56370163,  0.98619027]]),
 '4': array([[ 0.97914331,  0.53096623,  0.19786893,  0.34413299],
       [ 0.53096623,  0.92308924,  0.24814683,  0.31929757],
       [ 0.19786893,  0.24814683,  1.04500997,  0.50067075],
       [ 0.34413299,  0.31929757,  0.50067075,  0.98977

Um dicionário de tamanhos amostrais

In [25]:
node_sample_size

{'1': 120,
 '2': 120,
 '3': 240,
 '4': 300,
 'A': 60,
 'B': 60,
 'C': 60,
 'D': 60,
 'E': 60}

E um dicionário de médias.

In [26]:
node_means['1']

UMERO     7.856612
ULNA     15.933226
FEMUR    21.406435
TIBIA    28.609481
dtype: float64

Temos tb uma lista de todos os nós:

In [27]:
nodes

['4', 'E', '3', '2', 'C', 'B', '1', 'A', 'D']

É interessante notar como a matriz estimada para a raiz, ponderando as matrizes ao longo da filogenia, é idêntica à matriz ponderada dentro de grupos, calculada pelos modelos lineares anteriormente:

In [28]:
w_matrix

array([[ 0.97914331,  0.53096623,  0.19786893,  0.34413299],
       [ 0.53096623,  0.92308924,  0.24814683,  0.31929757],
       [ 0.19786893,  0.24814683,  1.04500997,  0.50067075],
       [ 0.34413299,  0.31929757,  0.50067075,  0.98977592]])

In [29]:
node_matrices['4']

array([[ 0.97914331,  0.53096623,  0.19786893,  0.34413299],
       [ 0.53096623,  0.92308924,  0.24814683,  0.31929757],
       [ 0.19786893,  0.24814683,  1.04500997,  0.50067075],
       [ 0.34413299,  0.31929757,  0.50067075,  0.98977592]])

# $\beta$ e $\Delta z$

Vamos agora estimar as mudanças evolutivas em cada ramo da filogenia, e, usando as matrizes ancestrais, calcular os gradientes de seleção estimados.

Os $\Delta z$ são apenas as diferenças nas médias de um nó com o seu acestral. Os $\beta$ são estimados com a equação de Lande:

$
\beta = G^{-1}\Delta z
$

In [21]:
delta_z = {}
beta = {}
for n in tree.nodes()[1:]: #começamos do 1 para pular a raiz, que não tem ancestral
    parent = get_node_name(n.parent_node)
    branch = get_node_name(n) + '_' + parent
    delta_z[branch] = node_means[get_node_name(n)] - node_means[parent]
    beta[branch] = np.linalg.solve(node_matrices[parent], delta_z[branch])

NameError: name 'tree' is not defined

In [31]:
delta_z

{'1_3': UMERO   -0.705685
ULNA    -1.352779
FEMUR   -1.079190
TIBIA   -1.308057
dtype: float64,
 '2_3': UMERO    0.705685
ULNA     1.352779
FEMUR    1.079190
TIBIA    1.308057
dtype: float64,
 '3_4': UMERO    0.233156
ULNA     0.575522
FEMUR    0.386529
TIBIA    0.477889
dtype: float64,
 'A_1': UMERO   -2.907894
ULNA    -5.984023
FEMUR   -6.295242
TIBIA   -8.429543
dtype: float64,
 'B_2': UMERO   -0.818866
ULNA    -1.595626
FEMUR    2.019179
TIBIA    2.814317
dtype: float64,
 'C_2': UMERO    0.818866
ULNA     1.595626
FEMUR   -2.019179
TIBIA   -2.814317
dtype: float64,
 'D_1': UMERO    2.907894
ULNA     5.984023
FEMUR    6.295242
TIBIA    8.429543
dtype: float64,
 'E_4': UMERO   -0.932624
ULNA    -2.302086
FEMUR   -1.546117
TIBIA   -1.911554
dtype: float64}

In [32]:
beta

{'1_3': UMERO    0.619133
ULNA    -1.381834
FEMUR   -0.298876
TIBIA   -0.854565
dtype: float64,
 '2_3': UMERO   -0.619133
ULNA     1.381834
FEMUR    0.298876
TIBIA    0.854565
dtype: float64,
 '3_4': UMERO   -0.226949
ULNA     0.617106
FEMUR    0.122174
TIBIA    0.300856
dtype: float64,
 'A_1': UMERO    5.593902
ULNA    -6.391306
FEMUR    0.724606
TIBIA   -8.174476
dtype: float64,
 'B_2': UMERO   -0.315532
ULNA    -2.334416
FEMUR    0.795698
TIBIA    3.166784
dtype: float64,
 'C_2': UMERO    0.315532
ULNA     2.334416
FEMUR   -0.795698
TIBIA   -3.166784
dtype: float64,
 'D_1': UMERO   -5.593902
ULNA     6.391306
FEMUR   -0.724606
TIBIA    8.174476
dtype: float64,
 'E_4': UMERO    0.907795
ULNA    -2.468425
FEMUR   -0.488695
TIBIA   -1.203423
dtype: float64}

Podemos agora calcular a correlação entre os $\beta$ e $\Delta z$

In [33]:
def vector_corr(x, y): return (np.dot(x, y)/(np.linalg.norm(x)*np.linalg.norm(y)))
corr_beta_delta_z = {}
for branch in delta_z:
    corr_beta_delta_z[branch] = vector_corr(beta[branch], delta_z[branch])

In [34]:
corr_beta_delta_z

{'1_3': 0.71383549975253691,
 '2_3': 0.71383549975253691,
 '3_4': 0.7698330333256046,
 'A_1': 0.5871776089626437,
 'B_2': 0.92344579494465995,
 'C_2': 0.92344579494465973,
 'D_1': 0.58717760896264337,
 'E_4': 0.76983303332560482}

Podemos também calcular a relação entre a resposta evolutiva e o primeiro componente principal da matriz de covariação, a linha de menor resistência evolutiva. 

Como a direção primeiro componente é arbitrária, tomamos o valor absoluto da correlação.

In [35]:
corr_pc1 = {}
for branch in delta_z:
    parent = branch.split("_")[1]
    pc1 = np.linalg.eig(node_matrices[parent])[1][:,0]
    corr_pc1[branch] = abs(vector_corr(delta_z[branch], pc1))

In [43]:
corr_pc1

{'1_3': 0.97603202614854889,
 '2_3': 0.97603202614854878,
 '3_4': 0.95783416390763121,
 'A_1': 0.95299530773479413,
 'B_2': 0.20772619269497655,
 'C_2': 0.2077261926949763,
 'D_1': 0.95299530773479413,
 'E_4': 0.95783416390763154}

Podemos utilizar o pandas para formatar esses resultados em tabelas

In [44]:
df_betas = pd.DataFrame.from_dict(beta, orient='index')
df_betas.columns = traits
df_betas

,UMERO,ULNA,FEMUR,TIBIA
1_3,0.619133,-1.381834,-0.298876,-0.854565
2_3,-0.619133,1.381834,0.298876,0.854565
3_4,-0.226949,0.617106,0.122174,0.300856
A_1,5.593902,-6.391306,0.724606,-8.174476
B_2,-0.315532,-2.334416,0.795698,3.166784
C_2,0.315532,2.334416,-0.795698,-3.166784
D_1,-5.593902,6.391306,-0.724606,8.174476
E_4,0.907795,-2.468425,-0.488695,-1.203423


In [45]:
df_dz = pd.DataFrame.from_dict(delta_z, orient='index')
df_dz.columns = traits
df_dz

,UMERO,ULNA,FEMUR,TIBIA
1_3,-0.705685,-1.352779,-1.079190,-1.308057
2_3,0.705685,1.352779,1.079190,1.308057
3_4,0.233156,0.575522,0.386529,0.477889
A_1,-2.907894,-5.984023,-6.295242,-8.429543
B_2,-0.818866,-1.595626,2.019179,2.814317
C_2,0.818866,1.595626,-2.019179,-2.814317
D_1,2.907894,5.984023,6.295242,8.429543
E_4,-0.932624,-2.302086,-1.546117,-1.911554


In [46]:
traits_c = list(traits)
traits_c.append('otu')
df_matrices = pd.DataFrame(columns=traits_c)
for node in node_matrices:
    df = pd.DataFrame(node_matrices[node], columns=traits, index = traits)
    df['otu'] = node
    df_matrices = df_matrices.append(df)
df_matrices


,UMERO,ULNA,FEMUR,TIBIA,otu
UMERO,0.856428,0.775719,0.216278,0.275482,A
ULNA,0.775719,1.086305,0.317295,0.357705,A
FEMUR,0.216278,0.317295,1.003959,0.796279,A
TIBIA,0.275482,0.357705,0.796279,1.067811,A
UMERO,0.985693,0.757909,0.016274,0.328934,C
ULNA,0.757909,0.936668,0.085370,0.336382,C
FEMUR,0.016274,0.085370,1.195272,0.173524,C
TIBIA,0.328934,0.336382,0.173524,0.911711,C
UMERO,1.089062,0.441458,0.173216,0.197140,B
ULNA,0.441458,0.953183,0.037394,0.137664,B
